In [1]:
import DCCRN
import torch
from asteroid.data import DNSDataset
import os
import time
import torch.nn as nn
import shutil
import numpy as np
from framework import ABF, MultiResolutionSTFTLoss, SPKDLoss, build_review_kd, ReviewKD
import asteroid
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import config as cfg
from tools_for_model import near_avg_index, max_index, min_index, Bar, cal_pesq, cal_stoi
from dataloader import create_dataloader
import inspect


/opt/conda/lib/python3.10/site-packages/asteroid/filterbanks/__init__.py:24: VisibleDeprecationWarning: asteroid.filterbanks has been replaced by `asteroid_filterbanks` and will be totally remove in a future release. Please use `asteroid_filterbanks` instead.
  warnings.warn(


In [2]:
model = DCCRN.DCCRN(rnn_units=cfg.rnn_units, masking_mode=cfg.masking_mode, use_clstm=cfg.use_clstm,
                  kernel_num=cfg.kernel_num)
#data_set =  DNSDataset("/root/NTH_student/Speech_Enhancement_new/asteroid/egs/dns_challenge/baseline/data")
train_dataset =  DNSDataset("/root/NTH_student/test_loader")
train_loader = create_dataloader(mode='train',dataset=train_dataset)

In [3]:
# tạo sample để input mode. cái này random đại thui
# input_size = model.stft.weight.size(1)
# tạo cái batch có n= 5 đi
# input = torch.randn(64, input_size)

In [4]:
# Tạo list chứa các feature map do encoder output ra 
# dùng list vì lát nữa feature map của lstm với decoder bỏ vô đây luôn. ở đây demo cái encoder thui nha. 
feature_maps = []
# Tạo hàm hook, hàm này sẽ thực thi khi encoder thực hiện forward.
# output của encoder sẽ được append vào trong encoder_feature_map(một biến toàn cục)
def encoder_hook(module, input, output):
    feature_maps.append(output)
# đăng kí forward hook. kiểu "ê khi nào encoder thực hiện foward xong thì chạy hàm encoder_hook nha"
model.encoder[-1].register_forward_hook(encoder_hook)

In [5]:
def decoder_hook(module, input, output):
    feature_maps.append(output)

model.decoder[-1].register_forward_hook(decoder_hook)

In [6]:
def enhance_hook(module, input, output):
    feature_maps.append(output)

model.enhance.register_forward_hook(enhance_hook)

In [7]:
X,Y = 0,0
i = 0
for x,y,_ in Bar(train_loader):
    if i == 1: break
    X=x
    Y=y
    i += 1

try :
    model(X)
    print('au de')
except:
    print('failed roihuhu')


 16/600: [>.................................................] - ETA 156.2s

/root/NTH_student/Speech_Enhancement_new/knowledge_distillation_CLSKD/tools_for_model.py:472: UserWarning: This overload of addcmul is deprecated:
	addcmul(Tensor input, Number value, Tensor tensor1, Tensor tensor2, *, Tensor out)
Consider using one of the following signatures instead:
	addcmul(Tensor input, Tensor tensor1, Tensor tensor2, *, Number value, Tensor out) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1174.)
  delta = torch.addcmul(Vrr * Vii, -1, Vri, Vri)


au de


In [8]:
len(feature_maps)

3

In [9]:
feature_maps[0].shape #[64, 256, 4, 483]


torch.Size([8, 256, 4, 483])

In [12]:
feature_maps[1][1].shape

torch.Size([483, 8, 512])

IndexError: list index out of range

In [ ]:
in_channels = [32, 64, 128, 256, 256, 256]
out_channels = [32, 64, 128, 256, 256, 256]

# shapes = [1,4,8,16,32]
# out_shapes = [1,4,8,16,32]
shapes = [4,4,4]
out_shapes = [4,4,4]
abfs = nn.ModuleList()
mid_channel = min(512, in_channels[-1])
for idx, in_channel in enumerate(in_channels):
    abfs.append(ABF(in_channel, mid_channel, out_channels[idx], idx < len(in_channels)-1))
abfs = abfs[::-1]

In [ ]:
student_features = model(X,is_feat=True)
student_features.shape

torch.Size([8, 514, 483])

In [ ]:
logit = student_features
x = feature_maps[0] #encoder
results = []
out_features, res_features = abfs[0](x, out_shape=out_shapes[0])
results.append(out_features)
for abf, shape, out_shape in zip(abfs[1:], shapes[1:], out_shapes[1:]):
        out_features, res_features = abf(x, res_features, shape, out_shape)
results.insert(0, out_features)

In [ ]:
in_channels = [32, 64, 128, 256, 64, 2]
out_channels = [32, 64, 128, 256, 64, 256]

# shapes = [1,4,8,16,32]
# out_shapes = [1,4,8,16,32]
shapes = [1,4,4,4,4]
#out_shapes = [1,4,4]
out_shapes = [4,4,4,4,4]
abfs = nn.ModuleList()
mid_channel = min(512, in_channels[-1])
for idx, in_channel in enumerate(in_channels):
    abfs.append(ABF(in_channel, mid_channel, out_channels[idx], idx < len(in_channels)-1))
abfs = abfs[::-1]

In [ ]:
student_features = model(X,is_feat=True)
logit = student_features[0]
x = feature_maps[1]
results = []
out_features, res_features = abfs[0](x, out_shape=out_shapes[0])
results.append(out_features)
for abf, shape, out_shape in zip(abfs[:1], shapes[:1], out_shapes[:1]):
        out_features, res_features = abf(x, res_features, shape, out_shape)
        results.insert(0, out_features)
results

[tensor([[[[ 4.8838e-02,  8.0968e-02,  1.8654e-01,  5.5018e-01],
           [-3.3673e-01, -9.9233e-01,  9.4533e-01, -1.3836e+00],
           [ 3.0834e-01, -5.2510e-01,  3.8587e-01, -4.7086e-01],
           [ 4.1400e-02, -1.4577e-01, -1.3671e-01,  1.3911e-01]],
 
          [[ 1.2012e-01,  1.0055e+00,  2.9543e-01, -1.6048e+00],
           [ 3.6149e-02,  4.3319e-01,  5.0801e-01, -1.0067e-01],
           [ 2.1605e-01, -4.6365e-01, -5.7357e-01,  1.7140e-01],
           [-3.0595e-01, -1.3363e-01, -1.0429e-01, -3.2293e-02]],
 
          [[ 2.6542e-01,  2.1241e-02, -1.3590e-01, -3.7505e-01],
           [ 1.9147e-01, -8.5183e-02, -8.9291e-01, -9.8409e-01],
           [ 5.3226e-01,  2.5964e-01, -1.5828e-01,  5.2180e-01],
           [ 3.5505e-01,  4.3134e-01,  2.8067e-01,  5.1049e-01]],
 
          ...,
 
          [[ 2.2907e-02,  1.0429e+00, -3.4651e-01, -7.8848e-01],
           [ 6.9959e-02,  9.6139e-01, -5.7878e-01, -1.1192e+00],
           [-4.7630e-01,  2.7079e-01, -3.9287e-01,  1.0199e-01],

In [ ]:
results[0].shape

torch.Size([8, 256, 4, 4])

In [ ]:
feature_maps[2].shape

torch.Size([8, 256, 4, 483])

In [ ]:
feature_maps[1][0].shape

torch.Size([2, 256, 484])

In [ ]:
feature_maps[1][1].shape

torch.Size([2, 256, 484])

In [ ]:
feature_maps.clear()